In [30]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import datetime
import pandas as pd
import numpy as np

In [31]:
#load the trained model, scaler and encoder
model = load_model('model.h5')
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('one_hot_encoder_geo.pkl', 'rb') as f:
    label_encoder_geo = pickle.load(f)


In [32]:
#example input data
input_data = {
    'CreditScore': 700,
    'Geography': 'France',
    'Gender' : 'Male',
    'Age': 30,
    'Tenure': 5,
    'Balance': 50000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}

In [33]:
# convert input data into categorical and numerical features
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\naresh\AiML\4-ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [34]:
#combine categorical and numerical features into input Data
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,France,Male,30,5,50000,2,1,1,60000


In [35]:
input_data = pd.concat([input_data_df.reset_index(drop=True), geo_encoded_df], axis=1)
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,France,Male,30,5,50000,2,1,1,60000,1.0,0.0,0.0


In [36]:
input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,France,1,30,5,50000,2,1,1,60000


In [37]:
## concatination with one hat encoded geo
input_data_df = pd.concat([input_data_df.drop('Geography', axis=1),geo_encoded_df], axis=1)

In [38]:
input_data_df 

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,1,30,5,50000,2,1,1,60000,1.0,0.0,0.0


In [39]:
#scale the input data
input_scaled = scaler.transform(input_data_df)
input_scaled

array([[ 0.50178794,  0.91324755, -0.84593077, -0.00134472, -0.41792108,
         0.80843615,  0.64920267,  0.97481699, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [44]:
#predict the output
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 16ms/step


array([[0.0129948]], dtype=float32)

In [45]:
#prediction probobabilities
prediction_proba = prediction[0][0]
prediction_proba

0.012994797

In [46]:
if prediction_proba > 0.5:
    print("The customer is likely to leave the bank.")
else:
    print("The customer is likely to stay with the bank.")

The customer is likely to stay with the bank.
